In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
# 전 결합층의 부모 클래스 
class BaseLayer:
  def update(self,eta): # 경사하강법 
    self.w -= eta * self.grad_w
    self.b -= eta * self.grad_b

In [3]:
# --은닉층-- 
class MiddleLayer(BaseLayer):
  def __init__(self,n_upper,n):
    # He 초깃값 사용 , n_upper : 앞 층의 뉴런 수 , n : 해당 층의 뉴런 수 
    self.w = np.random.randn(n_upper,n) * np.sqrt(2/n_upper)
    self.b = np.zeros(n)

  def forward(self,x): # 순전파 구현 
    self.x = x 
    self.u = np.dot(x, self.w) + self.b
    self.y = np.where(self.u <= 0 , 0, self.u ) # ReLU 함수 

  def backward(self,grad_y): # 역전파 구현 
    delta = grad_y * np.where(self.u<=0 , 0 , 0 , 1) # ReLU 의 편미분 값을 곱해줌 
    
    self.grad_w = np.dot(self.x.T, delta)
    self.grad_b = np.sum(delta,axis = 0) # 각 열에 있는 수를 더함 
    self.grad_x = np.dot(delta, self.w.T)

In [4]:
# 출력층 
class OutputLayer(BaseLayer):
  def __init__(self,n_upper,n):
    # 자비에르 초기화 기반의 초깃값 
    self.w = np.random.randn(n_upper,n) / np.sqrt(n_upper)
    self.b = np.zeros(n)
  
  def forward(self,x):
    self.x = x
    u = np.dot(x,self.w) + self.b
    # 소프트 맥스 함수 
    self.y = np.exp(u)/np.sum(np.exp(u), axis = 1, keepdims = True) # axis를 1로 지정해 각 샘플마다 총합을 구하고 keepdims = True 로 하여 배열의 차원을 유지 
    # self.y = np.exp(u-np.max(u))/np.sum(np.exp(u-np.max()), axis = 1 , keepdims = True) # 지수함수의 오버플로우를 막기 위한 코드 

  def backward(self,t):
    delta = self.y - t # 수식적으로 정리해낸 값, t는 각 출력에 대응하는 실제 정답이다. 

    self.grad_w = np.dot(self.x.T, delta)
    self.grad_b = np.sum(delta,axis = 0) 
    self.grad_x = np.dot(delta, self.w.T)
